In [1]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [2]:
#검색할 식당 데이터 임포트
rest_data = pd.read_csv('종로구.csv', encoding='utf-8-sig')
rest_data

,순번,resturant,최종 지역,합친 검색단어
0,0.0,누이네,광장시장,누이네 광장시장
1,1.0,자매집,광장시장,자매집 광장시장
2,8.0,온더보더,광화문,온더보더 광화문
3,9.0,도토리브라더스,광화문,도토리브라더스 광화문
4,13.0,버거파크,대학로,버거파크 대학로
...,...,...,...,...
498,NaN,NaN,NaN,NaN
499,NaN,NaN,NaN,NaN
500,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN


In [9]:
#결측치 처리 및 식당명만 변수 지정
rest_data=rest_data.dropna()
rest_data.info()

items = rest_data['합친 검색단어']
print(items)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 451 entries, 0 to 450
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   순번         451 non-null    float64
 1   resturant  451 non-null    object 
 2   최종 지역      451 non-null    object 
 3   합친 검색단어    451 non-null    object 
dtypes: float64(1), object(3)
memory usage: 17.6+ KB
0         누이네 광장시장
1         자매집 광장시장
2         온더보더 광화문
3      도토리브라더스 광화문
4         버거파크 대학로
          ...     
446          빚짜 종로
447        차니오뎅 혜화
448         홍곱창 혜화
449         도도야 혜화
450        까페랑솜 혜화
Name: 합친 검색단어, Length: 451, dtype: object


In [10]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(items) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    #카카오맵 접속 후 검색창에 입력
    driver.get("https://map.kakao.com/")
    
    #검색창에 식당명 입력   
    searchbox = driver.find_element(by='xpath', value="//input[@id='search.keyword.query']") 
    searchbox.send_keys(item)
    time.sleep(2)
    
  
    #검색 버튼 누르기
    searchbutton = driver.find_element(by='xpath', value="//button[@id='search.keyword.submit']") 
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    #검색한 식당이 있을 때 리뷰 정보 가져오기
    if len(driver.find_elements(by='xpath', value="//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        
        #리뷰수 버튼 클릭
        reviewbutton = driver.find_element(by='xpath', value="//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href")
        
        #리뷰창이 새 탭에서 뜨도록 처리 
        tabs = driver.window_handles
        driver.switch_to.window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(2)
        
        

        #스크롤 내리기
        for c in range(0,1):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
        
        
        #리뷰 데이터 스크래핑을 위한 html 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        
        #리뷰 수가 0이 아닌 경우 리뷰글 가져오기
        if len(review_lists) != 0 :
            
            #후기 더보기가 있으면 모두 눌러주기 
            while True: 
                try:
                    if driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.txt_more").textContent') == "후기 더보기":
                        driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.ico_comm.ico_more").click()')
                        time.sleep(2) 
                        
                except:
                    print("더보기 종료")
                    break 
                
            #후기 더보기를 모두 누른 후 리뷰 리스트 재정의
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            review_lists = soup.select('.list_evaluation > li')
            print(len(review_lists))
                    
            
            #리뷰 정보가 있는 경우 리뷰글, 식당 평균 평점, 리뷰 작성 시간 가져오기     
            try:
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.comment_info > p > span') # 리뷰
                        rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text #식당평균평점
                        timestamp = review.select(' div > span.time_write') #시간정보
                        
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        rev_list.append(
                        [
                            item,
                            rate,
                            user_review[0].text,
                            timestamp[0].text
                        ]
                        )            
    
                    time.sleep(1)
                
               
            except:
                print("더 이상 리뷰가 존재하지 않습니다.")
                
                
                
        else :
            #리뷰 정보가 없는 경우 식당명과 평균 평점만 데이터 프레임에 추가
            rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text
            rev_list.append(
                [
                    item,
                    rate,
                ]
            ) 

                    
            print("리뷰가 없습니다")
        
        #검색한 창 닫고 카카오맵 검색 페이지로 돌아가기    
        driver.close()
        driver.switch_to.window(tabs[0])
        print("기본 페이지로 돌아가기")
            
    else:
        print("식당이 존재하지 않음")
        
driver.close() 

진행상황 : 1/451
식당 존재
더보기 종료
50
기본 페이지로 돌아가기
진행상황 : 2/451
식당 존재
더보기 종료
107
기본 페이지로 돌아가기
진행상황 : 3/451
식당 존재
더보기 종료
56
기본 페이지로 돌아가기
진행상황 : 4/451
식당 존재
더보기 종료
169
기본 페이지로 돌아가기
진행상황 : 5/451
식당 존재
더보기 종료
130
기본 페이지로 돌아가기
진행상황 : 6/451
식당 존재
더보기 종료
149
기본 페이지로 돌아가기
진행상황 : 7/451
식당 존재
더보기 종료
19
기본 페이지로 돌아가기
진행상황 : 8/451
식당 존재
더보기 종료
214
기본 페이지로 돌아가기
진행상황 : 9/451
식당 존재
더보기 종료
100
기본 페이지로 돌아가기
진행상황 : 10/451
식당이 존재하지 않음
진행상황 : 11/451
식당 존재
더보기 종료
116
기본 페이지로 돌아가기
진행상황 : 12/451
식당 존재
더보기 종료


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CC1ED3+2236115]
	Ordinal0 [0x00C592F1+1807089]
	Ordinal0 [0x00B666FD+812797]
	Ordinal0 [0x00B4DFFA+712698]
	Ordinal0 [0x00BB506B+1134699]
	Ordinal0 [0x00BC514A+1200458]
	Ordinal0 [0x00BB18A6+1120422]
	Ordinal0 [0x00B8A73D+960317]
	Ordinal0 [0x00B8B71F+964383]
	GetHandleVerifier [0x00F6E7E2+2743074]
	GetHandleVerifier [0x00F608D4+2685972]
	GetHandleVerifier [0x00D52BAA+532202]
	GetHandleVerifier [0x00D51990+527568]
	Ordinal0 [0x00C6080C+1837068]
	Ordinal0 [0x00C64CD8+1854680]
	Ordinal0 [0x00C64DC5+1854917]
	Ordinal0 [0x00C6ED64+1895780]
	BaseThreadInitThunk [0x74DC7BA9+25]
	RtlInitializeExceptionChain [0x76F2BB3B+107]
	RtlClearBits [0x76F2BABF+191]


In [11]:
#스크래핑한 데이터를 데이터 프레임으로 만들기    
columns = ["name", "rate", "review", "timestamp"]
df = pd.DataFrame(rev_list, columns=columns)
df    

,name,rate,review,timestamp
0,누이네 광장시장,4.3,최고,2022.09.23.
1,누이네 광장시장,4.3,일행들과 별로 배가 안고프다며 갔다가 접시까지 싹싹 핥고 나온곳이예요일단 회전률이 ...,2022.09.10.
2,누이네 광장시장,4.3,푸짐하게 먹고나왔는데 영수증을 보니..통큰누이네가 아닌 광장시장순희네빈대떡이네요....,2022.09.07.
3,누이네 광장시장,4.3,빈대떡 바삭하고 두툼하고 맛있음. 정신은 좀 사나움,2022.09.03.
4,누이네 광장시장,4.3,녹두빈대떡이 바삭하고 맛있었어요,2022.08.29.
...,...,...,...,...
1105,김진목삼 서촌,4.5,너무 맛있어서 감동 ㅠ 정말 오랜만에 맛난 음식 대접 받은 느낌!! 강추,2019.01.17.
1106,김진목삼 서촌,4.5,,2018.10.31.
1107,김진목삼 서촌,4.5,사장님 친절하시고 맛있음!,2018.04.29.
1108,김진목삼 서촌,4.5,고기가 매우 맛있음.직접 구워주시고 친절함.자리가 협소한게 흠임.3테이블은 예약 4...,2018.01.19.


In [13]:
#데이터 프레임 전처리#

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

#중복값 제거
df = df.drop_duplicates(['review'], keep='first',ignore_index = True)

#리뷰글이 없는 결측치 제거
df['review']=df['review'].dropna()

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

쏘리에스프레소바 서촌    214
도토리브라더스 광화문    169
호호식당 대학로       149
버거파크 대학로       130
김진목삼 서촌        116
자매집 광장시장       107
공기식당 서촌        100
온더보더 광화문        56
누이네 광장시장        50
뮤통 서촌           19
Name: name, dtype: int64
쏘리에스프레소바 서촌    192
도토리브라더스 광화문    145
호호식당 대학로       112
버거파크 대학로       101
김진목삼 서촌         88
공기식당 서촌         79
자매집 광장시장        65
온더보더 광화문        49
누이네 광장시장        43
뮤통 서촌           14
Name: name, dtype: int64


C:\Users\soyoung\AppData\Local\Temp\ipykernel_5416\300514302.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].dropna()


In [14]:
#스크래핑한 데이터 프레임을 csv로 저장
df.to_csv('sample.csv', encoding='utf-8-sig')